In [1]:
import sys
sys.path.append('../')

import tquant as tq
from datetime import date

In [2]:
evaluation_date = tq.Settings.evaluation_date = date(2024,2,14)

# Fixed Flows

In [3]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
d1 = calendar.advance(evaluation_date,
                         6,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing)

In [4]:
nominal = 1e6
fixed_coupon = tq.FixedCoupon(d1,
                              nominal,
                              accrual_start_date= evaluation_date,
                              accrual_end_date= d1,
                              ref_period_start= evaluation_date,
                              ref_period_end= d1,
                              r= 0.02,
                              daycounter = daycounter)

In [5]:
coupon_df = fixed_coupon.display()
coupon_df

,start_period,end_period,payment_date,notional,accrual,day_counter,rate,amount
0,2024-02-14,2024-08-14,2024-08-14,1000000.0,0.505556,Actual360,0.02,10111.111111


In [6]:
coupon_df['amount']

0    10111.111111
Name: amount, dtype: object

In [7]:
coupon_df['rate'] * coupon_df['notional'] * coupon_df['accrual'] 

0    10111.111111
dtype: object

## Fixed Leg

In [8]:
fixed_schedule = [tq.TARGET().advance(evaluation_date,
                         t,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing) for t in [0,6,12,18,24]]
notionals = [1e6] * (len(fixed_schedule) - 1)
coupon_rates = [0.03] * (len(fixed_schedule) - 1)

In [9]:
fixed_leg = tq.FixedRateLeg(fixed_schedule,
                            notionals,
                            coupon_rates,
                            daycounter= daycounter,
                            compounding=tq.CompoundingType.Simple,
                            frequency= tq.Frequency.Annual)

In [10]:
fixed_leg.display_flows()

,start_period,end_period,payment_date,notional,accrual,day_counter,rate,amount
0,2024-02-14,2024-08-14,2024-08-14,1000000.0,0.505556,Actual360,0.03,15166.666667
0,2024-08-14,2025-02-14,2025-02-14,1000000.0,0.511111,Actual360,0.03,15333.333333
0,2025-02-14,2025-08-14,2025-08-14,1000000.0,0.502778,Actual360,0.03,15083.333333
0,2025-08-14,2026-02-16,2026-02-16,1000000.0,0.516667,Actual360,0.03,15500.0


# Floating Flows

In [11]:
index = tq.IborIndex('EUR6M',
             calendar,
             6,
             tq.TimeUnit.Months)
gearing = 1
spread = 0

floating_coupon = tq.FloatingCoupon(d1,
                                    nominal,
                                    evaluation_date,
                                    d1,
                                    index,
                                    gearing,
                                    spread,
                                    evaluation_date,
                                    d1,
                                    daycounter
                                    )

In [12]:
floating_coupon.display()

,start_period,end_period,payment_date,notional,fixing_date,fixing_days,index,accrual,in_arrears,gearing,spread,day_counter
0,2024-02-14,2024-08-14,2024-08-14,1000000.0,2024-02-14,None,EUR6M,0.505556,False,1,0,Actual360


## Floating Leg 

In [13]:
spreads = [0] * (len(fixed_schedule) - 1)
gearing = [1] * (len(fixed_schedule) - 1)
floating_schedule = fixed_schedule

In [14]:
floating_leg = tq.FloatingRateLeg(floating_schedule,
                               notionals,
                               gearing,
                               spreads,
                               index,
                               daycounter)

In [15]:
floating_leg.display_flows()

,start_period,end_period,payment_date,notional,fixing_date,fixing_days,index,accrual,in_arrears,gearing,spread,day_counter
0,2024-02-14,2024-08-14,2024-08-14,1000000.0,2024-02-14,None,EUR6M,0.505556,False,1,0,Actual360
0,2024-08-14,2025-02-14,2025-02-14,1000000.0,2024-08-14,None,EUR6M,0.511111,False,1,0,Actual360
0,2025-02-14,2025-08-14,2025-08-14,1000000.0,2025-02-14,None,EUR6M,0.502778,False,1,0,Actual360
0,2025-08-14,2026-02-16,2026-02-16,1000000.0,2025-08-14,None,EUR6M,0.516667,False,1,0,Actual360
